# UNR - FCEIA 
## Tecnicatura Universitaria en Programación 
### NLP: Trabajo Práctico N°1 

---

**Integrantes**
- López Ceratto, Julieta : L-3311/1
- Crenna, Giuliano : C-7438/1

# Importamos librerías necesarias

In [86]:
import os
import pandas as pd
import pickle

from typing import List, Dict, Any

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [27]:
import warnings
warnings.filterwarnings("ignore")

# Carga de datasets

In [2]:
JUEGOS_PATH: str = os.path.join(os.getcwd(), 'data', 'bgg_database.csv')
PELICULAS_PATH: str = os.path.join(os.getcwd(), 'data', 'IMDB-Movie-Data.csv')
LIBROS_PATH: str = os.path.join(os.getcwd(), 'data', 'dataset_libros.csv')

In [3]:
dataset_juegos: pd.DataFrame = pd.read_csv(JUEGOS_PATH)
dataset_peliculas: pd.DataFrame = pd.read_csv(PELICULAS_PATH)
dataset_libros: pd.DataFrame = pd.read_csv(LIBROS_PATH)

## Traducciones de Datasets

# Modelo de Análisis de Sentimientos

Creo un dataset sencillo para entrenar al clasificador.

In [72]:
ESTADOS_ANIMO_PATH: str = os.path.join(os.getcwd(), 'data', 'estados_de_animo.csv')

In [73]:
df_estados_de_animo: pd.DataFrame = pd.read_csv(ESTADOS_ANIMO_PATH)

X: pd.Series = df_estados_de_animo['prompt']
y: pd.Series = df_estados_de_animo['estado_animo']

Hacemos un split de los datos y creamos un pipeline de trabajo utilizando el clasificador **MultinomialNB**.

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo_animo = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [75]:
modelo_animo.fit(X_train, y_train)

y_pred = modelo_animo.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Alegre       1.00      0.96      0.98        79
 Melancólico       1.00      1.00      1.00        65
 Ni fu ni fa       0.96      1.00      0.98        76

    accuracy                           0.99       220
   macro avg       0.99      0.99      0.99       220
weighted avg       0.99      0.99      0.99       220



Creamos una función para clasificar el prompt del usuario.

In [76]:
def clasificar_animo(prompt_usuario: str) -> str:
    estado_animo_predicho = modelo_animo.predict([prompt_usuario])[0]
    
    return estado_animo_predicho

In [77]:
nuevo_prompt = "La vida no significa nada"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Ni fu ni fa


In [78]:
nuevo_prompt = "Me siento feliz"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Alegre


In [85]:
nuevo_prompt = "El vacio se siente en mi"

print(f"Estado de ánimo: {clasificar_animo(nuevo_prompt)}")

Estado de ánimo: Melancólico


Exporto el modelo

In [89]:
ESTADO_ANIMO_MODEL_PATH: str = os.path.join(os.getcwd(), 'models', 'modelo_estado_animo.pickle')

pickle.dump(modelo_animo, open(ESTADO_ANIMO_MODEL_PATH, 'wb'))

# Análisis Datasets

## Análisis Dataset Juegos
Como Vemos en el dataset de juegos no se presentan datos nulos.

In [6]:
dataset_juegos.isna().sum()

rank                0
game_name           0
game_href           0
geek_rating         0
avg_rating          0
num_voters          0
description         0
yearpublished       0
minplayers          0
maxplayers          0
minplaytime         0
maxplaytime         0
minage              0
avgweight           0
best_num_players    0
designers           0
mechanics           0
categories          0
dtype: int64

## Análisis Dataset Películas

In [7]:
dataset_peliculas.isna().sum()

Rank                  0
Title                 0
Genre                 0
Description           0
Director              0
Actors                0
Year                  0
Runtime (Minutes)     0
Rating                0
Votes                 0
Revenue (Millions)    0
Metascore             0
dtype: int64

## Dataset Libros
De los 5984 libros dentro del dataset, solo 53 tiene titulo secundario y solo 2796 libros tienen un autor registrado.

In [9]:
dataset_libros.shape

(5984, 4)

In [8]:
dataset_libros.isna().sum()

Titulo Principal        0
Titulo Secundario    5931
Autor                3188
N° Ref                  0
dtype: int64

# Apartado 1